In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import face_recognition
from PIL import Image
import numpy as np

In [3]:
mouthModel = keras.models.load_model("../cnn-mouth/base_cropped_yawning_model.h5")
eyesModel = keras.models.load_model("../cnn-eyes/base_eyes_model.h5")

In [4]:
def mouth_cropper(path):
    image = face_recognition.load_image_file(path)
    face_landmarks_list = face_recognition.face_landmarks(image)
    if face_landmarks_list:
        try:
            mouth = face_landmarks_list[0]['bottom_lip'] + face_landmarks_list[0]['top_lip']
        except KeyError:
            return None

        x_coords = [coord[0] for coord in mouth]
        y_coords = [coord[1] for coord in mouth]

        left = min(x_coords)
        top = min(y_coords)
        right = max(x_coords)
        bottom = max(y_coords)

        im = Image.open(path)
        im = im.crop((left, top, right, bottom))

        im = im.resize((80, 80))

        return im

In [5]:
def eye_cropper(path):
    image = face_recognition.load_image_file(path)
    face_landmarks_list = face_recognition.face_landmarks(image)
    eyes = []
    ans = []
    try:
        eyes.append(face_landmarks_list[0]['left_eye'])
        eyes.append(face_landmarks_list[0]['right_eye'])
    except:
        return None

    for eye in eyes:
        x_max = max([coordinate[0] for coordinate in eye])
        x_min = min([coordinate[0] for coordinate in eye])
        y_max = max([coordinate[1] for coordinate in eye])
        y_min = min([coordinate[1] for coordinate in eye])
        # establish the range of x and y coordinates    
        x_range = x_max - x_min
        y_range = y_max - y_min
        
        # to make sure the full eye is captured,
        # calculate the coordinates of a square that has 50%
        # cushion added to the axis with a larger range
        if x_range > y_range:
            right = round(.5*x_range) + x_max
            left = x_min - round(.5*x_range)
            bottom = round(((right-left) - y_range))/2 + y_max
            top = y_min - round(((right-left) - y_range))/2
        else:
            bottom = round(.5*y_range) + y_max
            top = y_min - round(.5*y_range)
            right = round(((bottom-top) - x_range))/2 + x_max
            left = x_min - round(((bottom-top) - x_range))/2
        
        #crop original image using the cushioned coordinates
        im = Image.open(path)
        im = im.crop((left, top, right, bottom))
        
        # resize image for input into our model
        im = im.resize((80,80))
        ans.append(im)

    return ans

In [35]:
# Test
root = "../data/test/mouth/"
total = 0
correct = 0
TP = 0
FP = 0
FN = 0

for folder in os.listdir(root):
    for im in os.listdir(root + folder):
        path = root + folder + "/" + im
        mouthImage = mouth_cropper(path)
        eyes = eye_cropper(path)
        drowsy = False

        if mouthImage:
            mouthImage = image.img_to_array(mouthImage)
            mouthImage = mouthImage/255.0
            mouthImage = np.expand_dims(mouthImage, axis=0)
            
            images = np.vstack([mouthImage])
            classes = mouthModel.predict(images)

            if classes[0]>0.5:
                drowsy = True

        if eyes:
            left, right = eyes[0], eyes[1]
            left, right = image.img_to_array(left), image.img_to_array(right)
            left, right = left/255.0, right/255.0
            left, right = np.expand_dims(left, axis=0), np.expand_dims(right, axis=0)
            left, right = np.vstack([left]), np.vstack([right])
            left, right = eyesModel.predict(left), eyesModel.predict(right)
            if left[0] > 0.5 and right[0] > 0.5:
                drowsy = True

        if drowsy and folder == "Yawning":
            correct += 1
            TP += 1
        elif not drowsy and folder == "Normal":
            correct += 1
        elif drowsy and folder == "Normal":
            FP += 1
        elif not drowsy and folder == "Yawning":
            FN += 1
        total += 1

../data/test/mouth/Normal/327867760_594764842492235_1868613635924958884_n.jpg
../data/test/mouth/Normal/328850083_731856778560861_9036283887921550214_n.jpg
../data/test/mouth/Normal/328908713_1401687160648422_2500127038507218711_n.jpg
../data/test/mouth/Normal/IMG-4711.jpg
../data/test/mouth/Normal/IMG-4712.jpg
../data/test/mouth/Normal/IMG-4713.jpg
../data/test/mouth/Normal/IMG-4714.jpg
../data/test/mouth/Normal/IMG-4715.jpg
../data/test/mouth/Normal/IMG-4716.jpg
../data/test/mouth/Normal/IMG-4727 (1).jpg
../data/test/mouth/Normal/IMG-4728.jpg
../data/test/mouth/Normal/IMG-4729.jpg
../data/test/mouth/Normal/IMG_0155.jpg
../data/test/mouth/Normal/IMG_0156.jpg
../data/test/mouth/Normal/IMG_0157.jpg
../data/test/mouth/Normal/IMG_0158.jpg
../data/test/mouth/Normal/IMG_0159.jpg
../data/test/mouth/Normal/IMG_0794.jpg
../data/test/mouth/Normal/IMG_0795.jpg
../data/test/mouth/Normal/IMG_0796.jpg
../data/test/mouth/Normal/Photo on 2023-01-31 at 12.46 PM #3.jpg
../data/test/mouth/Normal/Photo o

In [36]:
print(correct/total)

0.4838709677419355


In [7]:
# Test
root = "../data/archive/val/mouth/"
total = 0
correct = 0
TP = 0
FP = 0
FN = 0

for folder in os.listdir(root):
    for im in os.listdir(root + folder):
        path = root + folder + "/" + im
        print(path)
        mouthImage = mouth_cropper(path)
        eyes = eye_cropper(path)
        drowsy = False

        if mouthImage:
            mouthImage = image.img_to_array(mouthImage)
            mouthImage = mouthImage/255.0
            mouthImage = np.expand_dims(mouthImage, axis=0)
            
            images = np.vstack([mouthImage])
            classes = mouthModel.predict(images)

            if classes[0]>0.5:
                drowsy = True

        if eyes:
            left, right = eyes[0], eyes[1]
            left, right = image.img_to_array(left), image.img_to_array(right)
            left, right = left/255.0, right/255.0
            left, right = np.expand_dims(left, axis=0), np.expand_dims(right, axis=0)
            left, right = np.vstack([left]), np.vstack([right])
            left, right = eyesModel.predict(left), eyesModel.predict(right)
            if left[0] > 0.5 and right[0] > 0.5:
                drowsy = True

        if drowsy and folder == "yawn":
            correct += 1
            TP += 1
        elif not drowsy and folder == "no_yawn":
            correct += 1
        elif drowsy and folder == "no_yawn":
            FP += 1
        elif not drowsy and folder == "yawn":
            FN += 1
        total += 1

../data/archive/val/mouth/no_yawn/2008.jpg
../data/archive/val/mouth/no_yawn/2009.jpg
../data/archive/val/mouth/no_yawn/2010.jpg
../data/archive/val/mouth/no_yawn/2011.jpg
../data/archive/val/mouth/no_yawn/2012.jpg
../data/archive/val/mouth/no_yawn/2013.jpg
../data/archive/val/mouth/no_yawn/2014.jpg
../data/archive/val/mouth/no_yawn/2016.jpg
../data/archive/val/mouth/no_yawn/2031.jpg
../data/archive/val/mouth/no_yawn/2032.jpg
../data/archive/val/mouth/no_yawn/2033.jpg
../data/archive/val/mouth/no_yawn/2034.jpg
../data/archive/val/mouth/no_yawn/2056.jpg
../data/archive/val/mouth/no_yawn/2057.jpg
../data/archive/val/mouth/no_yawn/2058.jpg
../data/archive/val/mouth/no_yawn/2059.jpg
../data/archive/val/mouth/no_yawn/2062.jpg
../data/archive/val/mouth/no_yawn/2066.jpg
../data/archive/val/mouth/no_yawn/2067.jpg
../data/archive/val/mouth/no_yawn/2068.jpg
../data/archive/val/mouth/no_yawn/2069.jpg
../data/archive/val/mouth/no_yawn/2070.jpg
../data/archive/val/mouth/no_yawn/2071.jpg
../data/arc

In [8]:
print(correct/total)

0.6675461741424802
